## Рекуррентные нейронные сети. Генерация текстов. 
В этом домашнем задании вам предстоит применить на практике знания о рекуррентых сетях (в нашем случае - Long-Short Term Memory Network) в генеративном контексте. Это значит, что мы будем создавать модель, способную генерировать текст. 
Для обучения мы будем использовать роман «Преступление и наказание», который, благодаря своему немалому размеру, хорошо подойдет для обучения. 
Так же, это довольно интересно, насколько хорошо модель сможет заговорить языком великого писателя :)

Стоит заметить, что для обучения нейронных сетей требуются хорошие вычислительные мощности. Если вдруг вы сомневаетесь в своей GPU, то стоит обратить внимание на сервис Colaboratory (colab.research.google.com)  от Google. Этот сервис предоставляет мощную GPU на целых 12 часов бесплатно, при этом, если перезапускать ноутбук, то лимит на 12 часов каждый раз будет обновляться. Так же удобен тем, что не нужно никаких дополнительных настроек - просто загружаете свой ноутбук и начинаете работу. Во время разработки домашнего задания мы пользовались Colaboratory - и остались довольны. 

In [2]:
import sys
import io
import random
import pickle

from collections import Counter, defaultdict

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LambdaCallback, ModelCheckpoint

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
with io.open('book.txt', encoding='utf-8') as f:
    text = f.read().lower().replace('\xa0', ' ')
print('количество символов в корпусе:', len(text))

количество символов в корпусе: 1116083


Посмотрим на первые символы нашего корпуса

In [3]:
text[:200]

'роман ф. м. достоевского «преступление и наказание» принадлежит к числу тех великих произведений мировой литературы, ценность которых со временем не умаляется, но возрастает для каждого следующего пок'

### Maximum Likelihood Character Level Language Model
Прежде чем перейти к нейронным сетям, рассмотрим более простую генеративную языковую модель, которую чаще всего называют  Maximum Likelihood Character Level Language Model. 

Ее идея предельно проста: считаем условные вероятности для каждой буквы на обучающем множестве. Например: для буквы "д" считаем сколько раз встретились другие буквы после "д" в нашем датасете. Считаем сумму частот, делим на нее, частоту каждой буквы, получаем некоторое приближение вероятности. 

Реализуем данный подход!

In [4]:
class SimpleMaximumLikelihoodModel():
    def __init__(self, order=1):
        self.order = order
        self.model = None
        
    def fit(self, text):
        buffer = defaultdict(Counter)
        for i in range(len(text) - self.order):
            history, char = text[i:i + self.order], text[i + self.order]
            buffer[history][char] += 1
        self.model = {hist: self.normalize(chars) for hist, chars in buffer.items()}        
        
    def predict(self, symbol):
        if model is not None:
            return self.model[symbol]
        else:
            raise NotImplementedError()
            
    def normalize(self, counter):
        s = float(sum(counter.values()))
        return [(c, cnt / s) for c, cnt in counter.items()]

Построим модель 1-го порядка, т.е. считающую вероятность последующей буквы для каждой буквы. Модель 2-го порядка будет считать вероятность следующей буквы для пары букв, и так далее. 

In [5]:
order = 1

In [6]:
model = SimpleMaximumLikelihoodModel(order=order)
model.fit(text)

Посмотрим, какой наиболее вероятный символ после буквы "а" 

In [7]:
model.predict('а')

[('н', 0.05035312581742087),
 ('к', 0.08249774755137036),
 ('з', 0.051820850408347136),
 ('д', 0.02716017089545732),
 ('т', 0.0636643706222571),
 ('л', 0.10385967971633679),
 ('с', 0.06952073705931933),
 ('е', 0.019138547388612783),
 ('ж', 0.02374516813439126),
 ('м', 0.05054204086377772),
 ('у', 0.000988171011712733),
 ('р', 0.02621559566367309),
 ('в', 0.04099456505943558),
 (' ', 0.1907315371872003),
 ('.', 0.012570116546051675),
 ('х', 0.013543755631121574),
 ('п', 0.01153834975441044),
 (')', 0.0005376812857848693),
 ('б', 0.007367686807916994),
 (',', 0.03509460284244485),
 ('й', 0.013311244804836225),
 ('ч', 0.013718138750835586),
 ('я', 0.02915104484552562),
 ('и', 0.0017292992704972825),
 ('щ', 0.0037201732205655825),
 ('ф', 0.000755660185427384),
 ('ю', 0.013369372511407563),
 ('»', 0.0009736390850698986),
 ('ш', 0.013078733978550876),
 ('г', 0.009896242043770162),
 ('ц', 0.000755660185427384),
 ('…', 0.002979044961781033),
 (':', 0.0010753625715697387),
 (';', 0.001787426977

Попробуем что-нибудь сгенерировать с помощью модели первого порядка. 

Важный момент: при каждом запуске ячейки будет генерироваться новый набор символов, потому что мы не просто выбираем для каждой буквы последующую с максимальной вероятностью, а создаем вероятностное распределение, из которого генерируем новую букву.  

In [8]:
# количество букв в start_symbol должно быть больше или равно order
start_symbol = 'делов'
n_iter = 200
generated_string = start_symbol

for _ in range(n_iter):
    predictions = model.predict(generated_string[-order:])
    probabilities = np.random.multinomial(1, [x[1] for x in predictions], 1)
    sampled_letter = predictions[np.argmax(probabilities)][0]
    generated_string += sampled_letter
    
print(generated_string)

делов сяко рупрослстнакоришло? н в т азла льконакадаза, дорут чтахоська … брамем, вегок эть, омобя ра ро инарагам тето ны содово. вня вск. я визосвлой, жудези итех — за в.
ан диаедралейлю, зупралено обедут


Видно, что если строить модель по условной вероятности для одной буквы, то качество получается не очень хорошим. Попробуйте увеличить порядок (например до 5 или 7), и посмотрите, как изменится качество генерируемого текста. Модель уже способна генерировать осмыленные слова, и даже улавливать некоторую структуру

***Задание №1:*** какой символ имеет наибольшую вероятность для строки "здоро", если обучить модель порядка 5?
В EdX отправьте символ в нижнем регистре

In [9]:
# ваше решение
order = 5
model = SimpleMaximumLikelihoodModel(order=order)
model.fit(text)
print(model.predict('здоро'))

# генерируем текст
start_symbol = 'здоро'
n_iter = 200
generated_string = start_symbol

for _ in range(n_iter):
    predictions = model.predict(generated_string[-order:])
    probabilities = np.random.multinomial(1, [x[1] for x in predictions], 1)
    sampled_letter = predictions[np.argmax(probabilities)][0]
    generated_string += sampled_letter
    
print(generated_string)

[('в', 1.0)]
здоровы…

— не имея, так чтоб опасался он к свидеть образом составить, — показать близких и так, приобретается как-нибудь; приду, ну, сажая его глаз, ведь этой злобная и, так и вчера, вчера я, может быть о


После того, как вы ознакомились с более простой языковой моделью, которая, как вы заметили, работает не так уж плохо, время переходить к нейронным сетям. Начнем с важного шага - предобработки данных. 

#### Предобработка текста

In [6]:
chars = sorted(list(set(text)))
print('количество уникальных символов в тексте:', len(chars))

количество уникальных символов в тексте: 95


In [7]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Сформируем выборку для обучения модели. В качестве Х мы будем использовать часть текста (т.е. набор символов), ограниченную длиной max_length, а в качестве Y - следующий символ. Затем закодируем каждый символ символ в Х с помощью бинарного вектора. 

step - это шаг, с которым мы идем по нашему корпусу, разбивая его на элементы обучающей выборки. 

In [8]:
max_length = 70
step = 5
sentences = []
next_chars = []

for i in range(0, len(text) - max_length, step):
    sentences.append(text[i: i + max_length])
    next_chars.append(text[i + max_length])
    
print('количество последовательностей:', len(sentences))

количество последовательностей: 223203


In [13]:
print('пример одного элемента обучающей выборки (X и y):\n', sentences[700], next_chars[700])

пример одного элемента обучающей выборки (X и y):
 орые полагали, что страдания и нищета неизбежны во всяком обществе, чт о


In [14]:
x = np.zeros((len(sentences), max_length, len(chars)), dtype=np.int8)
y = np.zeros((len(sentences), len(chars)), dtype=np.int8)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

#### Архитектура модели, выбор алгоритма оптимизации

Создаем многослойную рекуррентную сеть с двумя LSTM-слоями. Обратите внимание на входную размерность (input_shape): на вход мы принимаем кусок текста, длина которого равна max_length, каждый элемент которой имеет размерность len(chars) - это наш бинарный вектор, говорящий о том, какая это буква. 

Далее добавляем полносвязный слой (Dense), которой в этом случае будет выходом сети, с размерностью len(chars), так как мы хотим предсказать следующий символ, а так же с функцией активации Softmax - которая преобразует вектор так, что сумма всех его элементов равна 1 и имеет вероятностную интерпретацию.  

**Задание**: Добавьте второй LSTM-слой со 128 скрытыми параметрами. Добавьте Dropout слой с вероятностью отключения нейрона 0.5

In [26]:
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(max_length, len(chars))))
model.add(LSTM(128)) # ваш код
model.add(Dropout(0.5)) # ваш код
model.add(Dense(len(chars), activation='softmax'))

В качестве алгоритма подбора параметров модели выберем Adam ([Документация](https://keras.io/optimizers/), [Оригинал статьи](https://arxiv.org/abs/1412.6980v8)), немного изменив его стандартные параметры, а в качестве функции ошибки возьмем многоклассовую энтропию, т.к. предсказываем мы следующую букву, и должны делать это как можно точнее. 

In [27]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.99, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#### Реализуем вспомогательные функции для генерации текста с помощью модели
Ниже представлен код функций, которые упрощают генерацию текста с помощью модели. Генерировать слова и символы можно с помощью нескольких основных стратегий:
1. Greedy - выбираем символ с максимальной вероятностью из выхода сети (softmax)
2. Sampling - генерируем символ из вероятностного распределения на выходе сети (softmax)
3. Beam Search - более сложный алгоритм, учитывающий условные вероятности, позволяющий выбрать наиболее вероятную последовательность символов. 

В данном случае мы будем использовать стратегию №2, сэмплируя символ из распределения, которое получается на выходе из слоя Softmax. Результат Softmax'a можно интерпретировать вероятностно, т.к. после применения этой функции к вектору все его компоненты суммируются в единицу. Получив результат Softmax, мы создаем мультиномиальное распределение, из которого и генерируем символ. Важное значение имеет параметр temperature - чем он больше, тем более равномерным становится наше распределение. 
Вы можете увидеть, как меняются сгенерированные предложения с изменением этого параметра. 

In [12]:
def sample(predictions, temperature=1.0):
    """
    Функция для генерация слова из распределения; аргумент temperature позволяет
    сглаживать распределение. Чем его значение больше, тем более сглаженным
    получается распределение над символами. 
    """
    predictions = np.asarray(predictions).astype('float64')
    predictions = np.log(predictions) / temperature
    exp_predictions = np.exp(predictions)
    predictions = exp_predictions / np.sum(exp_predictions)
    probabilities = np.random.multinomial(1, predictions, 1)
    return np.argmax(probabilities)

def on_epoch_end(epoch, _):
    """
    Данная функция вызывается на каждой эпохе обучения сети. Ее задача - показать
    результаты модели. 
    """
    print()
    print('Генерируем текст после эпохи #%d' % epoch)

    start_index = random.randint(0, len(text) - max_length - 1)

    sentence = text[start_index: start_index + max_length]
    print('Предложение, на основании которого мы генерируем: ', sentence)

    for temperature in [0.05, 0.2, 0.5]:
        print('temperature = ', temperature)

        sentence = text[start_index: start_index + max_length]
        generated = sentence
        sys.stdout.write(generated)

        for i in range(200):
            x_pred = np.zeros((1, max_length, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            predictions = model.predict(x_pred, verbose=0)[0]
            next_index = sample(predictions, temperature)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

Аргумент callbacks принимает на вход функции, которые срабатывают в конце каждой эпохи. В нашем случае таких функций две:

**LambdaCallback**, который вызывает on_epoch_end, позволяющий генерировать нам предложение на каждой эпохе и наблюдать за поведением модели. 

**ModelCheckpoint** - сохраняет копию модели после каждой итерации на жесткий диск. 

In [62]:
history = model.fit(
    x, y,
    batch_size=128,
    epochs=1,
    callbacks=[
        LambdaCallback(on_epoch_end=on_epoch_end),
        ModelCheckpoint('2layers-weights-{epoch:02d}-{loss:.3f}.hdf5', monitor='loss')
    ]
)

223104/223203 [============================>.] - ETA: 0s - loss: 2.1809 - acc: 0.3600
Генерируем текст после эпохи #0
Предложение, на основании которого мы генерируем:  ая, неуклюжая, робкая и смиренная девка, чуть не идиотка, тридцати пят
temperature =  0.05
ая, неуклюжая, робкая и смиренная девка, чуть не идиотка, тридцати пять не соня и водел в соненно в соненно в соненно в столоников подомонить и не подомонить и всё подомал он подоловал он не соня стороников подомонить и подомонить и подомонить и всем не подомить и всё п
temperature =  0.2
ая, неуклюжая, робкая и смиренная девка, чуть не идиотка, тридцати пять не вого в соворять на столоников в соненно в соленить с продом столоников подомой подомил он подоление стал и подном подомонов сонение и всё сомнать и всё поделать полодать и водал на домоно не подо
temperature =  0.5
ая, неуклюжая, робкая и смиренная девка, чуть не идиотка, тридцати пять и ов раду колько не не семенной только на день говор! с навал даме вавуе и вакал и подру

#### Применение LSTM для генерации символов в тестовых данных

Мы подготовили для вас набор небольших предложений, в котором необходимо предсказать следующий символ для каждого предложения. Храниться этот набор в файле test_X.pickle

In [63]:
with open('test_X.pickle', 'rb') as f:
    test_X = pickle.load(f)

Код для предсказания следующего символа для каждого предложения с помощью вашей модели. Попробуйте варьировать параметр temperature, и то, как изменяется результат от него. 

In [75]:
answers = []
temperature = 0.5

for test_x in test_X:
    test_x_matrix = np.zeros((1, len(test_x), len(chars)))
    for t, char in enumerate(test_x):
        test_x_matrix[0, t, char_indices[char]] = 1.

    predictions = model.predict(test_x_matrix, verbose=0)[0]
    next_index = sample(predictions, temperature)
    next_char = indices_char[next_index]
    answers.append(next_char)

/home/kalan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


**Задание №2:** Данную строку необходимо отправить в качестве ответа в EdX. Ваша задача - обучить модель так, чтобы метрика accuracy превысила 0.35. Чтобы получить полный балл - необходимо получить хотя бы 0.38

In [76]:
answer_string = ','.join(answers)
print(answer_string)

 ,д,ь,а,д,р,а,и,ж,п,о, ,и,о, ,о,т,с,о,р,ч,т,л,с,о,о,в,о,в,с,ч,о,б,е,е,о,т,о,в,с,н,и,о,—,а,л, ,с,т, ,г,ж,е,н,т,в,в,д,е,п,л,с,е, ,е,к,л,о,д,т,ы,о,а,о,е,а,и,а,и,л,а,т,о,м,п,т,е,ь,е,в,у,и,о,л,ь,п,с,ь,е,е


## Генерация предложений с помощью обученной модели
В этом разделе мы загрузим предобученную модель для генерации текста, и посмотрим, как она работает. 

In [9]:
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(max_length, len(chars))))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(len(chars), activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
model.load_weights('checkpoint.hdf5')

Вы можете изменять переменную sentence, и наблюдать, какой текст получается на выходе. Заметим, что модель, обученная на уровне символов, научилась генерировать целые осмысленные слова, улавливая долгосрочные взаимосвязи. 

Можете попробовать подвигать значение температуры, и посмотреть, как меняются в зависмости от нее результаты модели. Так же можно менять исходное предложение, но важно помнить, что его длина должна быть равна max_length

In [16]:
temperature = 0.5

sentence = 'этот факт был тщательно расследован и довольно хорошо засвидетельствов'
generated = sentence
sys.stdout.write(generated)

assert len(sentence) == max_length, 'Длина sentence должна быть равна max_length'

for i in range(120):
    x_pred = np.zeros((1, max_length, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.

    predictions = model.predict(x_pred, verbose=0)[0]
    next_index = sample(predictions, temperature)
    next_char = indices_char[next_index]

    generated += next_char
    sentence = sentence[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()

этот факт был тщательно расследован и довольно хорошо засвидетельствовала. тол

/home/kalan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


ько не смешал и скорее его поверение, который дом. выходите, по праве. подостовали его полезывать к тему дело оч